In [7]:
import tensorflow as tf
import os
import keras.backend as K
import pandas as pd
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LeakyReLU, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D, MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import imutils
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import argparse
import imutils
import time
import cv2

In [8]:
airplanes = pd.read_csv(r"C:\Users\Carlos M\Documents\Machine Learning\Bound Box Regressor\bounding-box-regression\dataset\airplanes.csv")
airplanes

,Unnamed: 0,Xmin,Ymin,Xmax,Ymax
0,image_0001.jpg,49,30,349,137
1,image_0002.jpg,59,35,342,153
2,image_0003.jpg,47,36,331,135
3,image_0004.jpg,47,24,342,141
4,image_0005.jpg,48,18,339,146
...,...,...,...,...,...
795,image_0796.jpg,57,27,356,118
796,image_0797.jpg,56,25,350,110
797,image_0798.jpg,59,25,354,110
798,image_0799.jpg,49,26,347,116


In [9]:
DATADIR = r"C:\Users\Carlos M\Documents\Machine Learning\Bound Box Regressor\bounding-box-regression\dataset\images"
CATEGORIES = ["airplanes"]
training_data = list()
Xmin = airplanes["Xmin"].to_numpy()
Ymin = airplanes["Ymin"].to_numpy()
Xmax = airplanes["Xmax"].to_numpy()
Ymax = airplanes["Ymax"].to_numpy()
width = list()
length = list()
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, img))
                img_array = img / 255.0
                img_width = img_array.shape[0]
                img_length = img_array.shape[1]
                width.append(img_width)
                length.append(img_length)
                new_array = cv2.resize(img_array, (224, 224))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()
X = list()
Y = list()
for features, label in training_data:
    X.append(features)
    Y.append(label)

width = np.array(width)
length = np.array(length)
    
X = np.array(X).reshape(-3, 224, 224, 3)
print(X.shape)
Y = np.array(Y)

ratio_Xmin = np.divide(Xmin, length)
ratio_Ymin = np.divide(Ymin, width)
ratio_Xmax = np.divide(Xmax, length)
ratio_Ymax = np.divide(Ymax, width)
# Were training the CNN on the ratio of value of Xmin, Ymin, Xmax, Ymax
# aka bound box points in relation to the image width and length

(800, 224, 224, 3)


In [10]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=64,activation="relu"))
model.add(Dense(units=32, activation="relu"))
model.add(Dense(units=4,activation="sigmoid"))


opt = Adam(lr=0.001)
model.compile(loss="mse", optimizer=opt)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [11]:
data = zip(ratio_Xmin, ratio_Ymin, ratio_Xmax, ratio_Ymax)
data = list(data)
airplanes["data"] = data
data = np.array(data)
airplanes

,Unnamed: 0,Xmin,Ymin,Xmax,Ymax,data
0,image_0001.jpg,49,30,349,137,"(0.12311557788944724, 0.18292682926829268, 0.8..."
1,image_0002.jpg,59,35,342,153,"(0.14713216957605985, 0.19021739130434784, 0.8..."
2,image_0003.jpg,47,36,331,135,"(0.11959287531806616, 0.21818181818181817, 0.8..."
3,image_0004.jpg,47,24,342,141,"(0.11959287531806616, 0.1411764705882353, 0.87..."
4,image_0005.jpg,48,18,339,146,"(0.12151898734177215, 0.10227272727272728, 0.8..."
...,...,...,...,...,...,...
795,image_0796.jpg,57,27,356,118,"(0.1386861313868613, 0.1875, 0.866180048661800..."
796,image_0797.jpg,56,25,350,110,"(0.1382716049382716, 0.18382352941176472, 0.86..."
797,image_0798.jpg,59,25,354,110,"(0.14425427872860636, 0.18796992481203006, 0.8..."
798,image_0799.jpg,49,26,347,116,"(0.12311557788944724, 0.18439716312056736, 0.8..."


In [20]:
model.fit(X, data, epochs = 1, validation_split = 0.3)

Train on 560 samples, validate on 240 samples
560/560 [==============================] - 135s 241ms/sample - loss: 0.0304 - val_loss: 0.0288


In [21]:
imagepath = r"C:\Users\Carlos M\Pictures\airplane.jpg"
image = cv2.imread(imagepath)
image = image / 255.0
image = cv2.resize(image, (224, 224))
print(image.shape)
image = np.expand_dims(image, axis = 0)
preds = model.predict(image)[0]
(startX, startY, endX, endY) = preds
startX = int(startX * 224)
startY = int(startY * 224)
endX = int(endX * 224)
endY = int(endY * 224)
print([startX, startY, endX, endY])

(224, 224, 3)
[0, 0, 224, 224]


In [22]:
image = cv2.imread(imagepath)
image = cv2.resize(image, (224, 224))

In [23]:
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()  